# Essential Tools and Libraries

* **IPython Notebook**


>Kernel: IPython is an alternative Python command line shell for interactive computing with lots of useful enhancements over the "default" Python interpreter.
<br><br>
>Jupyter: The browser-based documents Jupyter Notebook is a great environment for scientific computing: Not only to execute code, but also to add informative documentation via Markdown, HTML, LaTeX, embedded images, and inline data plots.

* **numpy**

>NumPy, short for Numerical Python. It provides the data structures, algorithms, and library glue needed for most scientific applications involving numerical data in Python. 
<br><br>
>Beyond the fast array-processing capabilities that NumPy adds to Python, one of its primary uses in data analysis is as a container for data to be passed between algorithms and libraries.


* **Matplotlib**

>Matplotlib is the most popular Python library for data visualizations which can generate plots, histograms, power spectra, bar charts, error charts, scatter plots, etc., with just a few lines of code. It will helps us to make easy things easy and hard things possible.


* **Seaborn**

>Seaborn is a library for making attractive and informative statistical graphics in Python. It aims to make visualization a central part of exploring and understanding data. And it is built on top of matplotlib and tightly integrated with the PyData stack, including support for numpy and pandas data structures and statistical routines from scipy and statsmodels.

* **Scikit-learn**

>Scikit-learn is is the most popular general machine library for Python. It includes a broad range of different classifiers, cross-validation and other model selection methods, dimensionality reduction techniques, modules for regression and clustering analysis, and a useful data-preprocessing module.

* **Scikit-image**

>Scikit-image is a collection of algorithms for image processing.

* **pip**

>The PYPA recomneded tool for installing Python packages

* * Install LPP package

>$ pip install lpproj 

# Introduction

Face recognition is one of many applications of digital image processing. It is concerned with the automatic identification of an individual in a digital image. There are many algorithms through which this process can be carried out. One of these algorithms compresses a database of face images and keeps only the data useful for facial approximation. Any face image can then be approximated using only the information contained in the compressed database, even if the image was not in the original database.

In my project, using 3 techniques: Principal Component Analysis(PCA), Linear Discriminant Analysis(LDA) and Locality Preserving Projection(LPP). By using LPP, the face images are mapped into a face subspace for analysis. Different from PCA and LDA which effectively see only the Euclidean structure of face space, LPP finds an embedding that preserves local information, and obtains a face subspace that best detects the essential face manifold structure. A face subspace is obtained by Locality Preserving Projection(LPP), each face image in the image space is mapped to a low-dimensional face subspace, which is characterized by a set of feature images, called Laplacianfaces. Face recognition by projecting face images onto a feature space called “eigenfaces” through PCA. LDA, on the other hand, is a supervised learning algorithm. The aim of LDA is  to find a linear combination of features which separate different classes are far away from each other while requiring objects in the same class are close to each other.



However, both PCA and LDA only effectively see the Euclidean Structure. They fail to discover the underlying structure which might be a nonlinear submanifold. In 2005, He, Yan, Hu, Niyogi and Zhang proposed a new approach which considers the manifold structure for face analysis through Locality Preserving Projection(LPP). They used LPP to find the face subspaces which they called as Laplacianfaces to achieve face recognition.


In this project, the above three methods have been successfully implemented. After the system is trained by the training data, the feature space “eigenfaces” through PCA, the feature space “fisherfaces” though LDA and the feature space “laplacianfaces” through LPP are found using respective methods. Then use the different classifier SVM and KNN to calculate the accuracy, and analysis the result.

# Database

This project using Scikit-learn dataset which is "fetch_lfw_people". This dataset is a collection of JPEG pictures of famous people collected over the internet, all details are available on the official website: http://vis-www.cs.umass.edu/lfw/

## Load the dataset

In [2]:
from sklearn.datasets import fetch_lfw_people

#Download the data, if not already on disk and load it as numpy array
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

#introspect the images arrays to find the shapes(for plotting)
n_samples, h, w = lfw_people.images.shape

#for machine learning we use the 2 data directly(as relative pixel
#positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" %n_features)
print("n_classes: %d" %n_classes)

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


From the output we can see, in this situation, the dataset totally have 1288 samples, and each sample have 1850 features

## Spliting the dataset as training and testing

In [14]:
from sklearn.model_selection import train_test_split
#split into a training set and a test set using a stratified k fold

#split into a tranining adn testing set

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 42)

print("Training data size: "+ str(X_train.shape[0]))
print("Testing data size: "+str(y_test.shape[0]))

Training data size: 1030
Testing data size: 258


# Principal Component Analysis(PCA)

Let $X = \{x_1,x_2,...,x_n\}$ be the matrix containing n face images. Note that each image matrix has been converted into a vector. For example, a $m*n$ matrix is converted into a vector with $m*n$ rows. The subspace “eigenfaces” could be obtained by following the below steps:

&emsp; 1. Computer the mean $\mu$
\begin{equation}
    \mu = \frac{1}{n}\sum_{n}^{i=1}x_i
\end{equation}

&emsp; 2. Compute the Covariance Matrix S
\begin{equation}
    S = \frac{1}{n}\sum_{n}^{i=1}(x_i-\mu)(x_i-\mu)^T
\end{equation}

&emsp; 3. Compute the eigenvectors $\nu_i$ and eigenvalues $\lambda_i$
\begin{equation}
    S\nu_i = \lambda_i\nu_i
\end{equation}

&emsp; 4. Order the eigenvectors descending by their eigenvalues and select k eigenvectors corresponding to k largest eigenvalues

The above eigenvectors form the subspace , and it is called as “eigenfaces”.  There is still a computational problem left. For images in Sklearn(fetch_lfw_peopledd) database, each image is 50 x 37 meaning that there are 1850 dimensions. Once such faces are applied to calculate the covariance S, S will end up with a 1850 x 1850 matrix which is really huge and almost could not be processed on normal laptops. Here we firt define Y:
\begin{equation}
    Y = X -U
\end{equation}

where $U=\{\mu, \mu,...,\mu\}$. By this way, the original data set is transformed into a data set with zero mean. In order to solve the problem just mentioned, the eigenvectors and eigenvalues of$Y^TY$ is first calculated:
\begin{equation}
    Y^TY\nu_i = \lambda_i\nu_i
\end{equation}

Then the original eigenvectors are calculaed by a left multiplication of the data matrix Y:
\begin{equation}
    YY^T(Y\nu_i) = \lambda_i(Y\nu_i)
\end{equation}

The last step is to normalize the eigenvectors just calculated. Finally, the k eigenvectors corresponding to k largest eigenvalues are retrieved and form the subspace W. Also, the column vectors of W are the so-called eigenfaces.

Normally, the number of training samples is far less than the dimensions of each face. As a result, the above method is needed to avoid the computational problem mentioned before.

## How to implements it in python

Scikit-learn has a subpackage for  PCA. Linear dimensionality reduction using Singular Value Decompostion of the data to project it to a lower dimensional space.

In [16]:
from sklearn.decomposition import PCA

n_components = 150
pca = PCA(n_components = n_components, svd_solver='randomized',
          whiten = True).fit(X_train)

# Linear Discriminant Analysis(LDA)

LDA is a supervised learning algorithm meaning that the class labels of training set will be used in the training process. Let X be the matrix containing training face images and $X_i$ be the matrix containing face images belonging to class $i$.
\begin{equation}
    X=\{X_1,X_2, ..., X_C\}\\
    X_i = \{x_1, x_2, ... , x_n\}
\end{equation}

The scatter matrices $S_B$ and $S_w$ are calculated as follows:
\begin{equation}
    S_B = \sum_{i=1}^{c}N_i(\mu_i-\mu)(\mu_i-\mu)^T\\
    S_w = \sum_{i=1}^{c}\sum_{x_j\in x_i}(x_j-\mu_i)(x_j-\mu_i)^T
\end{equation}

where $N_i$ represents the number of training samples belonging to class $i$, $\mu_i$ represents the mean of training samples belonging to class label $i$ and $\mu$ and represents the mean of the total samples.

In order to find a subspace where $S_B$ is maximized and $S_w$ is minimized, the following general eigenvalue problem is needed to be solved.
\begin{equation}
    S_B\nu_i = \lambda_i S_w \nu_i
\end{equation}

However, one is confronted with the difficulty that the within-class scatter matrix $S_w$ is always singular. In order to solve this problem, it is suggested that PCA first projects the image set to a lower dimensional space so that the resulting within-class matrix $S_w$ is no longer singular. After this process, the general eigenvalue problem becomes:
\begin{equation}
    S_w^{-1} S_B\nu_i = \lambda_i\nu_i
\end{equation}

By solving the above eigenvalue problem $W_{fld}$, is obtained which defines a subspace. Please note that there are at most c-1 nonzero generalized eigenvalues for the above formula. Also, note that PCA can directly reduce the dimension of the feature space to N-c because the rank of $S_w$ is at most N-c . Here is N the number of samples while c is the number of classes.

By combing the transformation matrix of PCA, the below transformation matrix for LDA is obtained:
\begin{equation}
    W=W_{pca}W_{fld}
\end{equation}

The colum vectors of W are the so-called fisherfaces.

## How to implements it in python

Scikit-learn has a subpackage for LDA. A classifier with a linear decision boundary, generated by fitting class conditional densities to the data and using Bayes’ rule.

The model fits a Gaussian density to each class, assuming that all classes share the same covariance matrix.

The fitted model can also be used to reduce the dimensionality of the input by projecting it to the most discriminative directions.

In [ ]:
from sklearn.discriminat_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis(n_components = 150)
clf.fit(X_train, y_train)

# Locality Preserving Projections(LPP)

First, let's define some notation to make the rest of the tutorial easier to follow. Each sample in the dataset will be denoted by $\mathbf{x}_i$. It is common in derivations to consider data points to be represented by column vectors, but it is more natural in programming the algorithms to denote single data points as arrays, which end up as rows in a matrix of we put many of them together. The focus here will be more on programming than the underlying math, so we'll assume our dataset consists of $m$ points in $n$-dimensional space, forming a $m \times n$ matrix $X$.

The steps to perform LPP are as follows:

## Step1: PCA projection

The training image set is first projected into the PCA subspace by
throwing away several components with small eigenvalues. The reason of PCA projection is to ensure $XDX^T$ is not singular which is helpful to solve a generalized
eigenvector problem later on.

## Step2: Construct the Adjacency Graph

An adjacency graph simply describes which input samples are connected. Inputs that are similar should be connected (i.e. they are neighbors), and inputs that are dissimilar should not be connected. A simple way to represent the adjacency graph is an $m \times m$ matrix, where $m$ is the number of samples you have. If input $\mathbf{x}_i$ is connected to $\mathbf{x}_j$, the entry of the matrix at row $i$ and column $j$ is $1$, otherwise it is $0$. The matrix is then symmetric (if $\mathbf{x}_i$ is connected to $\mathbf{x}_j$, then $\mathbf{x}_j$ is connected to $\mathbf{x}_i$) and it is likely sparse (depending on how neighbors are determined).

He and Niyogi describe two methods of determining which samples are close to one another:

* $\epsilon$-neighborhoods: nodes $i$ and $j$ are connected if $||x_i-x_j||_2^2<\epsilon$
* $k$ nearest neighbor: nodes $i$ and $j$ are connected if $i$ is one of $k$ nearest neighbors of $j$ or vice versa (neighbors determined by Euclidean distance)

The authors also note that adjacency can be determined using other (less principled) methods and that LPP will attempt to preserve those neighborhoods.

scikit-learn has a subpackage (neighbors) for creating graphs and such. This makes it really easy to construct either of these adjacency graphs. By default, we'll cheat and have the adjacency matrices represent the connections by their Euclidean distance, unless mode='connectivity' is passed in. This will make step 3 easier in some cases.

In [ ]:
from sklearn import neighbors

def radius_adj(X, radius, mode='distance'):
    A = neighbors.radius_neighbors_graph(X, radius, mode=mode)
    return A

def kneighbors_adj(X, n_neighbors, mode='distance'):
    A = neighbors.kneighbors_graph(X, n_neighbors, mode=mode)
    return A

## Step3: Choose Weights

Since the adjacency graph specifies only which nodes are connected, we use a separate matrix to represent how strongly the connected nodes are connected. Like the adjacency graph, the weights are represented by an $m \times m$ matrix $W$. Again, the authors offer two options for determining the weights:

* Heat kernel: $W_{ij} = \exp \left( \frac{||\mathbf{x}_i - \mathbf{x}_j ||^2_2}{t} \right)$ if nodes $i$ and $j$ are connected and zero otherwise

* Simple: $W_{ij} = 1$ if nodes $i$ and $j$ are connected and zero otherwise

For the heat kernel, we'll rely on the ability of the graph constructors of scikit-learn to return the distances between neighbors. This makes the weight calculations simpler.

For the simple weight option, pass mode='connectivity' to the adjacency matrix construction function and the resulting weight matrix will be the same as the adjacency matrix.



In [ ]:
import numpy as np

def heat_kernel_weights(dists, param):
    W = -dists**2/param
    np.exp(W.data, W.data)
    return W

## Step 4: Compute the Eigenmapping

The final step involves solving the generalized eigen-problem:

$$
X^T L X \mathbf{a} = \lambda X^T D X \mathbf{a}
$$
Where $D$ is the diagonal matrix whose elements are the column-wise sums of $W$, and $L = D - W$ is the graph Laplacian. The nice thing here is that the products of matrices on both sides produce symmetric matrices, so we can use algorithms that are tuned for solving the eigenvalue problem for symmetric matrices.



The matrix of eigenvectors (i.e. the matrix whose columns are the eigenvectors arranged in order by increasing eigenvalue) then give us the transformation matrix to take inputs and transform them to the lower dimensional space. That is, we use the first $l$ eigenvectors (first meaning those with the smallest eigenvalues) to perform an embedding to $l$-dimensional space.



In [ ]:
from scipy.linalg import eigh

def compute_mapping(W, l):
    D = np.diagflat(W.sum(axis=0))
    L = D - W
    eigvals, eigvecs = eigh(X.T.dot(L).dot(X),
                            X.T.dot(D).dot(X),
                            eigvals=(0, l-1))
    return eigvecs

## Puting it Together

We'll put things together to make a simple interface to LPP.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class LocalityPreservingProjection(BaseEstimator, TransformerMixin):
    
    def __init__(self, n_components=2, adjacency='kneighbors',
                 adjacency_param=3, weights='heatkernel',
                 kernel_param=0.1):
        self.n_components = n_components
        self.adjacency = adjacency
        self.adjacency_param = adjacency_param
        self.weights = weights
        self.kernel_param = kernel_param
        
    def fit(self, X, y=None):
        if self.adjacency == 'kneighbors':
            adj_func = kneighbors_adj
        else:
            adj_func = radius_adj
            
        if self.weights == 'heatkernel':
            mode = 'distance'
        else:
            mode = 'connectivity'
            
        W = adj_func(X, self.adjacency_param, mode=mode)
        
        if self.weights == 'heatkernel':
            W = heat_kernel_weights(W, self.kernel_param)
        
        self.components_ = compute_mapping(W, self.n_components)
        
        return self
            
    def transform(self, X):
        return X.dot(self.components_)

Basic example as follow:

In [ ]:
from lpproj import LocalityPreservingProjection 

lpp = LocalityPreservingProjection(n_components=150, n_neighbors = 15)
lpp_train = lpp.fit_transform(X_train_pca)

# Support Vector Machines Classificaiton Algorithm

A Support Vector Machine (SVM) is a discriminative classifier formally defined by a separating hyperplane. In other words, given labeled training data (supervised learning), the algorithm outputs an optimal hyperplane which categorizes new examples.


Learning the SVM can be formulated as an optimization as following:
\begin{equation}
    min_w||w||^2 subject~ to ~y_i(w^Tx_i+b)\ge 1 ~for~ i =1...N
\end{equation}

This is a quadratic optimization problem subject to linear constraints and there is a unique minimum

## How to implements it in python

Scikit-learn has a subpackage for SVM

The advantages of support vector machines are:

* Effective in high dimensional spaces.
* Still effective in cases where number of dimensions is greater than the number of samples.
* Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
* Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

* If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.
* SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation.

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)

## How to calculate the accuracy using SVM

Base on above code for fit the svm model, after that we use test data to test the accuarcy.

In [ ]:
from sklearn.metrics import accuracy_score

svm_pred = clf.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred);

# K-Nearest Neigbor Classification Algorithm

The k-nearest neighbor algorithm is a method for classifying objects based on closest training objects: an object is classified according to a majority vote of its neighbors. In this project, KNN plays an important role in recognition phase. The steps of performing KNN are as follows:

## Step 1: find k nearest neighors

For a specific test face, it is first projected onto the subspace defined by W, and W is calculated by any methods mentioned above such as PCA, LDA and LPP. Afterward, an array with size is created to keep the K nearest neighbors, and the way to achieve this goal is to go through the whole training set and update the array if there is a training face whose distance is smaller than the largest distance in the original array. At the end of this process, the K nearest neighbors are found. Also, please notice that the training set has already been projected onto the same subspace defined by W. 

## Step 2: Classify according to weights.

After the K nearest neighbors are found, a HashMap is created to keep the pairs <label, weight> by going through all the neighbors. If the HashMap happens to meet a new label, <label, 1 / distance> is directly added into the map. Otherwise, an updated version which adds the original weight with 1 / distance is put into the map. After the HashMap is correctly constructed, this program will go through the whole HashMap and find the label associated with the largest weight. This class label will be the label recognized by this system.

## How to implement it in python

Scikit-learn has a subpackage for knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = NNeighborsClassifiter(n_neighbors=3)
neigh.fit(X_train, y_train)

## How to calcualte the accuracy using KNN

Base on above code for fit the knn model, we use test data to calculate the accuracy

In [ ]:
from sklearn.metrics import accuracy_score

knn_pred = neigh.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_pred)

# Result

After implement the project, we can get the accuracy as following:

|   |PCA|LDA|LPP|
|----|----|----|----|
|SVM|88%|61%|17%|
|KNN| 66%|79%|46%|


From the table we can see Eigenfaces(PCA) has a best the performance under SVM classifier model, and Laplacianfaces(LPP) has really low accuracy under SVM model. Under the KNN classifier model, LPP has a better performance than SVM model, but still not good enough. On the contrary, LDA under KNN model, has a better performance.

Then if we change the number of the training set, lets see how the accuracy look like:

|Training set|PCA|LDA|LPP|
|-----|-----|-----|-----|
|90% data as training|0.67|0.66|0.51|
|80% data as training|0.62|0.68|0.46|
|70% data as training|0.49|0.71|0.42|
|60% data as training|0.49|0.72|0.42|
|50% data as training|0.46|0.74|0.36|

>Note: here we use KNN classifier model to caclculate the accuracy when to change to training set size, since KNN is fast, the result of deviation for each algorithm is not too high.

From the table, we can see when the training set decreasing, the accuracy for PCA and LPP is decreasing, but LDA is increasing. From the LDA algorithm, as we know, LDA need both X train and y train parameters to fit the algorithm which is different with PCA and LPP, both algorithm only need X train data.

>Note: Since choosing the training data is randomized, but size is fixed. Therefore every time run the project will get the different accuracy for each algorithm, but the value is not changed too much and the trend should be same as we analyzed.

# Conclusion and furtherwork

## Conclusion

* From the result, we can find, PCA performs the best on Scikit-learn database if using SVM classifier model.

* LDA performs the best on Scikit-learn database if using KNN classifier model.

* LPP does not perform as expected. This method still could achieve good results with proper setting.

## Furtherwork

* Change the database to compare each algorithm

* Use of sophisticated and better distance metrics like variance normalized distance may improve the recognition performance.

# Reference

1. https://github.com/ixjlyons/sklearn-lpp/blob/master/lpp.ipynb

2. https://github.com/wihoho/FaceRecognition/blob/master/doc/FaceRecognitionReport.pdf

3. He, X., Yan, S., Hu, Y., Niyogi, P. and Zhang, H.J. (2005). Face Recognition Using
Laplacianfaces. IEEE Transactions on Pattern Analysis and Machine Intelligence, vol.
27, no. 3, pp. 328-340.